<a href="https://colab.research.google.com/github/dinesh-umkc/kdm/blob/main/Fake%20News%20Detection%20ICP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Objective
Fake news detaction

Try below vectorization method
1. Count vectorization
2. Word Level tf-idf
3. Ngram level tf-ifd  
4. Character tf-idf
5. Hashing Vector

Use below Classifiers

1. Logistic regression
2. Navie Bayes
3. SVM  


